In [ ]:
%matplotlib widget
from circuit import *
from solver import *

from models import NPN, PNP, Diode, Resistor, VoltageSource, CurrentSource, OpAmp
import sympy as sp

In [ ]:
subcircuit2 = Circuit()

Vin  = subcircuit2.addPort()
V1 = subcircuit2.addNode()
Vout = subcircuit2.addPort()

D1, R1, R2 = subcircuit2.addElements(
                        Diode("D1", (Vin, V1), Vd=0.6),
                        Resistor("R1", (V1, Vout), R=1e3),
                        Resistor("R2", (Vout, Gnd), R=1e3),
                        ).values()

V2, V3 = subcircuit2.addNodes(2)

2


{'V1'}

In [3]:
D1.Vs

[P0, V0]

In [7]:
circuit = Circuit()

var = Symbol("Vin", real=True)

Vcc = 5
R = 10

V1, V2 = circuit.addNodes(2)

circuit.addElements(
                    VoltageSource("Vg", (V1, Gnd), Vdc = var),             
                    )

circuit.addSubcircuit(subcircuit2, "Sub1", {'Vin' : V1, 'Vout' : V2})
circuit.addSubcircuit(subcircuit2, "Sub2", {'Vin' : V1, 'Vout' : V2})

circuit.compile()

{'variables': {I_Sub1_D1_0,
  I_Sub1_D1_1,
  I_Sub1_R1_0,
  I_Sub1_R1_1,
  I_Sub1_R2_0,
  I_Sub1_R2_1,
  I_Sub2_D1_0,
  I_Sub2_D1_1,
  I_Sub2_R1_0,
  I_Sub2_R1_1,
  I_Sub2_R2_0,
  I_Sub2_R2_1,
  I_Vg_0,
  I_Vg_1,
  V0,
  V3,
  V4},
 'equations': {Eq(I_Sub1_D1_0 + I_Sub2_D1_0 + I_Vg_0, 0),
  Eq(I_Sub1_D1_1 + I_Sub1_R1_0, 0),
  Eq(I_Sub1_R1_1 + I_Sub1_R2_0 + I_Sub2_R1_1 + I_Sub2_R2_0, 0),
  Eq(I_Sub2_D1_1 + I_Sub2_R1_0, 0)},
 'nodes': {'V3': <node.Node at 0x19fc2cada50>,
  'V4': <node.Node at 0x19fc2cae510>,
  'Sub1_V1': <node.Node at 0x19fc2c617d0>,
  'Sub2_V1': <node.Node at 0x19fc2caf4d0>},
 'elements': {'Vg': <models.VoltageSource at 0x19fc2cae890>,
  'Sub1_D1': <models.Diode at 0x19fc2caec50>,
  'Sub1_R1': <models.Resistor at 0x19fc2caf190>,
  'Sub1_R2': <models.Resistor at 0x19fc2caf210>,
  'Sub2_D1': <models.Diode at 0x19fc2caf410>,
  'Sub2_R1': <models.Resistor at 0x19fc2caf850>,
  'Sub2_R2': <models.Resistor at 0x19fc2caf950>}}

In [5]:
solver = Solver(circuit)
model = solver.solveDC(debugLog=True)

{I_Vg_1, I_Sub1_D1_1, I_Sub1_R2_1, I_Sub1_D1_0, I_Sub1_R1_1, I_Vg_0, I_Sub1_R1_0, V2, V0, V1, I_Sub1_R2_0}
{Eq(I_Sub1_D1_1 + I_Sub1_R1_0, 0), Eq(I_Sub1_R1_1 + I_Sub1_R2_0, 0), Eq(I_Sub1_D1_0 + I_Vg_0, 0)}
{'Vg': <models.VoltageSource object at 0x0000019FC1F5D4D0>, 'Sub1_D1': <models.Diode object at 0x0000019FC25E42D0>, 'Sub1_R1': <models.Resistor object at 0x0000019FC20FB290>, 'Sub1_R2': <models.Resistor object at 0x0000019FC073F010>}
[Eq(I_Sub1_D1_1 + I_Sub1_R1_0, 0), Eq(I_Sub1_R1_1 + I_Sub1_R2_0, 0), Eq(I_Sub1_D1_0 + I_Vg_0, 0), Eq(V1, Vin), Eq(I_Vg_0 + I_Vg_1, 0), Eq(I_Sub1_D1_0, 0), Eq(I_Sub1_D1_1, 0), Eq(I_Sub1_R1_0, 0.001*V0 - 0.001*V2), Eq(I_Sub1_R1_0 + I_Sub1_R1_1, 0), Eq(I_Sub1_R2_0, 0.001*V2), Eq(I_Sub1_R2_0 + I_Sub1_R2_1, 0)]
[-V0 + V1 < 0.6]
{'Sub1_D1': 'Cut'}
{I_Sub1_D1_1: 0.0, I_Sub1_R1_0: 0.0, I_Sub1_R1_1: 0.0, I_Sub1_R2_0: 0.0, I_Sub1_R2_1: 0.0, V0: 0.0, V2: 0.0, I_Sub1_D1_0: 0.0, I_Vg_0: 0.0, I_Vg_1: 0.0, V1: Vin}
[Vin < 0.6]
Interval.open(-oo, 0.600000000000000)
-----

In [6]:
Solver.printModel(model, var)

-------------------------------------
{'Sub1_D1': 'Cut'}
Vin ∈ Interval.open(-oo, 0.600000000000000)
{I_Sub1_D1_1: 0.0, I_Sub1_R1_0: 0.0, I_Sub1_R1_1: 0.0, I_Sub1_R2_0: 0.0, I_Sub1_R2_1: 0.0, V0: 0.0, V2: 0.0, I_Sub1_D1_0: 0.0, I_Vg_0: 0.0, I_Vg_1: 0.0, V1: Vin}
-------------------------------------
-------------------------------------
{'Sub1_D1': 'Fwd'}
Vin ∈ Interval(0.600000000000000, oo)
{I_Sub1_D1_0: 0.0005*Vin - 0.0003, I_Sub1_D1_1: 0.0003 - 0.0005*Vin, I_Sub1_R1_0: 0.0005*Vin - 0.0003, I_Sub1_R1_1: 0.0003 - 0.0005*Vin, I_Sub1_R2_0: 0.0005*Vin - 0.0003, I_Sub1_R2_1: 0.0003 - 0.0005*Vin, I_Vg_0: 0.0003 - 0.0005*Vin, I_Vg_1: 0.0005*Vin - 0.0003, V0: Vin - 0.6, V1: Vin, V2: 0.5*Vin - 0.3}
-------------------------------------
***************************************************************************************************************


In [ ]:
from sympy import diff

Id = circuit.currentThroughElement("Sub1_D1")

measurments =   [   
                    (lambda sol : V2().subs(sol), "Vout"),
                    (lambda sol : Id.subs(sol), "Id"),
                ]

fig, ax = plt.subplots(figsize=[11, 7])
plotMeasurments(model, -10, 10, 0.1, measurments, var)
plt.legend(loc="best");
plt.grid(True);
